Dieses Notebook enthält eine kommentierte Version der Datei *bundesscraper.py*.

Import aller benötigten Module und Einstellen der Sprache für die Erkennung von Datumsangaben.

In [ ]:
import sys
import argparse
import logging
import re
import time
import datetime
import csv
from collections import namedtuple
# Required for parsing German dates
import locale
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')

from lxml import html

Es ist gute Praxis, beim Crawlen einer Seite kleine Pausen einzulegen, damit der Server nicht zu stark beeinträchtigt wird. Das Interval wird hier auf zwei Sekunden festgelegt.

In [ ]:
PAUSE = 2

Ein typischer Fall beim Crawlen ist, dass es zwei Typen von Seiten gibt: Eine **Übersichtsseite**, die eine Liste von Suchtreffern, Artikeln, o.ä. enthält, und **Detailseiten**, die zu jedem einzelnen dieser Objekte ausführliche Angaben liefern. Eine Suche in einem Online-Shop wird z.B. eine Liste von passenden Artikeln aufführen, während ein Klick auf einen einzelnen Artikel zu einer Seite mit weiteren Angaben, Bildern und Bewertungen führt.

Oft ist die Übersichtsseite in der Praxis dabei auf mehrere Seiten aufgeteilt, die jeweils eine bestimmte Anzahl von Treffern enthalten. Über eine Blätterfunktion kann man dann weitere Treffer aufrufen, etwa:

| 1 | 2 | 3 | 4 | nächste Seite |

Die Hauptfunktion ist `crawl()`. Sie nimmt zwei Parameter entgegen: `start_url` ist die erste Übersichtsseite, `outfile` ist der Name der Datei, in der das Ergebnis gespeichert werden soll. Ein typischer Aufruf sieht also so aus:

```python
crawl('http://www.bundesregierung.de/SiteGlobals/Forms/Webs/Breg/Suche/DE/Nachrichten/Redensuche2_formular.html',
      'reden.csv')
```

Die Funktion ruft dabei weitere Funktionen auf. Die Grundlogik ist dabei:

1. Alle Links zu Detailseiten finden: `find_links()`
2. Für jeden dieser Links die Detailseite aufrufen und die relevanten Daten extrahieren: `scrape_page()`
3. Das Ergebnis in eine CSV-Datei speichern: `save()`

In [ ]:
def crawl(start_url, outfile):
    """
    Crawl a list page, extract details and save results as CSV.
    
    """
    links = find_links(start_url)
    pages = [scrape_page(link) for link in links]
    save(pages, outfile)

Die Funktion `find_links()` dient dazu, alle Übersichtsseiten durchzugehen und die jeweiligen Links zu den Detailseiten in einer gemeinsamen Liste zusammenzuführen. Dabei guckt die Funktion für jede Übersichtsseite, ob sie einen Link auf eine weitere Übersichtsseite enthält. Falls ja, wird diese weitere Seite ebenfalls abgerufen, andernfalls ist die Liste vollständig und wird zurückgegeben.

In [ ]:
def find_links(start_url):
    """
    Find links to details pages from all subsequent list pages.
    
    """
    links = []
    next_page = start_url
    while next_page:
        new_links, next_page = scrape_list_page(next_page)
        links.extend(new_links)
    return links

Die Funktion `scrape_list_page()` ist dafür zuständig, eine konkrete Übersichtsseite abzurufen. Sie sucht dabei nach zwei Elementen, die sie anschließend zurückgibt:

1. Die Links zu den Detailseiten, und
2. Den Link zur nächsten Übersichtsseite. Gibt es keinen solchen Link, wird `None` zurückgegeben.

In [ ]:
def scrape_list_page(url):
    """
    Extract links to details pages from a specific list page.
    
    """
    # Wait
    time.sleep(PAUSE)
    # Get and parse page
    doc = html.parse(url)
    root = doc.getroot()
    root.make_links_absolute(url)
    # Extract links to details pages
    link_elements = root.cssselect('#searchResults h3 a')
    links = [element.get('href') for element in link_elements]
    # Extract link to next page
    next_link_elements = root.cssselect('.forward a')
    if next_link_elements:
        next_page = next_link_elements[0].get('href')
    else:
        next_page = None
    # Return
    return (links, next_page)

Die Funktion `scrape_page()` durchsucht eine Detailseite und extrahiert die relevanten Informationen. Für jede Seite wird dabei nach diesen Angaben gesucht, die als Liste zurückgegeben werden: URL, Titel, Datum, Ort, Abtract und Text.

In [ ]:
def scrape_page(url):
    """
    Scrape information from a details page.
    
    """
    # Wait
    time.sleep(PAUSE)
    # Get and parse page
    doc = html.parse(url)
    root = doc.getroot()
    # Title
    title = root.cssselect('#main h1')[0].text_content()
    # Metadata fields: Date and place
    date = None
    place = None
    metadata = root.cssselect('#main dl')
    if metadata:
        metadata = metadata[0]
        keys = metadata.cssselect('dt')
        values = metadata.cssselect('dd')
        for key, value in zip(keys, values):
            if key.text_content().strip(':') == 'Datum':
                date = parse_date(value.text_content())
            elif key.text_content().strip(':') == 'Ort':
                place = value.text_content()
    # Abstract and text
    abstract = root.cssselect('#main .abstract')
    if abstract:
        abstract = abstract[0].text_content()
    else:
        abstract = None
    paragraphs = root.cssselect('#main .basepage_pages > p')
    paragraphs = [p.text_content() for p in paragraphs]
    text = '\n\n'.join(paragraphs)
    # Strip superfluous session ID
    if ';jsessionid' in url:
        url = url.split(';jsessionid')[0]
    # Return
    return [url, title, date, place, abstract, text]

Die Funktion `parse_date()` ist der Übersichtlichkeit halber aus der Funktion `scrape_page()` ausgelagert. Sie dient dazu, Datumsangaben in ein standardisiertes Format zu überführen. Dabei werden zwei Formen von Datumsangaben erkannt: *2. November 2014* und *2.11.2014*. Beide Formen werden in der Ergebnisdatei als *2014-11-02* gespeichert.

Beide Datumsvarianten sind in der Variablen `date_patterns` in zwei Formen gespeichert: Einmal als regulärer Ausdruck, und einmal als dazugehöriges Datumsmuster. Mit dem regulären Ausdruck wird überprüft, ob das Datum im Text in der jeweiligen Form vorliegt. Falls ja, wird das entsprechende Datumsmuster zum Parsen verwendet.

In [ ]:
def parse_date(date_string):
    """
    Parse dates from a string and return a `datetime.date()` object.
    
    """
    date_string = date_string.strip()
    date_patterns = [
        (r'\d+\.\d+\.\d+', '%d.%m.%Y'),
        (r'\d+\. \w+ \d+', '%d. %B %Y')
    ]
    date = None
    for regex, pattern in date_patterns:
        match = re.match(regex, date_string)
        if match:
            # Limit date string to part that matches the regex.
            # Prevents parsing errors with additional data like " 17:00 Uhr"
            date_string = match.group(0)
            date = datetime.datetime.strptime(date_string, pattern).date()
    return date

Die Funktion `save()` speichert das Ergebnis als CSV-Datei. Der Parameter `pages` ist dabei eine Liste von Seiten, wobei jede Seite wieder eine Liste von URL, Titel, etc. ist.

In [ ]:
def save(pages, outfile):
    """
    Save list of pages to a CSV file.
    
    """
    with open(outfile, 'w', newline='') as csvfile:   
        writer = csv.writer(csvfile)
        writer.writerow(['url', 'title', 'date', 'place', 'abstract', 'text'])
        for page in pages:
            writer.writerow(page)

Bislang wurden nur die einzelnen Funktionen definiert. Um nun zu crawlen, wird die Hauptfunktion aufgerufen:

In [ ]:
START_URL = 'http://www.bundesregierung.de/SiteGlobals/Forms/Webs/Breg/Suche/DE/Nachrichten/Redensuche2_formular.html?nn=8988&searchtype.HASH=e5e7611fd92022248dd0&path.HASH=631e69fa04dc338f202c&path=%2Fbpainternet%2Fcontent%2Fde%2Frede*+%2Fbpainternet%2Fcontentarchiv%2Fde%2Farchiv17%2Frede*&doctype=speech&doctype.HASH=fb7e3f87bcd598d5d3d&searchtype=news'
OUTFILE = 'reden.csv'
crawl(START_URL, OUTFILE)

Der Durchgang kann dabei sehr lange dauern. Die Datei *bundesscraper.py* gibt daher regelmäßig Zwischeninformationen aus, um das Arbeiten des Programms nachvollziehen zu können.